In [6]:
pip install datasets

In [7]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [8]:
import os
import pandas as pd
import numpy as np
import re

In [9]:
# Imported the dataset
from datasets import load_dataset
from datasets import Dataset

train_set1=load_dataset('ai4privacy/pii-masking-300k', split='train')
val_set=load_dataset('ai4privacy/pii-masking-300k', split='validation')

In [10]:
# Set up train_set2
filepath = os.path.join(os.getcwd(), 'updated_combined_mini_df.csv')
train_df2 = pd.read_csv(filepath)
train_set2 = Dataset.from_pandas(train_df2)

In [11]:
# Inspect the dataset
print(type(train_set2))
print(train_set2.shape)
val_set.shape

<class 'datasets.arrow_dataset.Dataset'>
(200, 5)


(47728, 9)

In [12]:
import random
# Select 100 random indices
# random_indices = random.sample(range(len(train_set1)), 10)
random_indices = random.sample(range(len(train_set2)), 10)

# Get the random samples
# random_samples = train_set1.select(random_indices)
random_samples = train_set2.select(random_indices)

print(random_samples)

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'unmasked_text', 'masked_text', 'privacy_mask'],
    num_rows: 10
})


In [13]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('lakshyakh93/deberta_finetuned_pii')
list(tokenizer.vocab.items())[:10]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

[('translation', 48235),
 ('ĠJeremy', 5653),
 ('none', 39763),
 ('ĠPeg', 32517),
 ('328', 35252),
 ('Jar', 44768),
 ('Ġunless', 3867),
 ('ĠReceived', 45913),
 ('Ġ1600', 25183),
 ('Ġprope', 39238)]

In [14]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import evaluate
import torch

# Define and load the tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained('lakshyakh93/deberta_finetuned_pii')
# train_set1_selected = train_set1.select(range(5))

# Define label mapping
# label_map = {'O': 0, 'TIME': 1, 'DATE': 2, 'LASTNAME1': 3, 'LASTNAME2': 4, 'EMAIL': 5, 'SOCIALNUMBER': 6}
label_map = {'O': 0, 'FIRSTNAME_1': 1, 'LASTNAME_1': 2, 'EMAIL_1': 3, 'PHONENUMBER_1': 4, 'SSN_1': 5, 'JOBTITLE_1': 6,
             'SEX_1': 7, 'BUILDINGNUMBER_1': 8, 'STREET_1': 9, 'DOB_1': 10, 'USERNAME_1': 11, 'AGE_1': 12,
             'PREFIX_1': 13, 'ACCOUNTNUMBER_1': 14}


# Function to tokenize and align labels
def tokenize_and_align_labels_train(example):
    tokenized_inputs = tokenizer(example['unmasked_text'], truncation=True, padding='max_length', max_length=512)
    labels = ['O'] * len(tokenized_inputs['input_ids'])  # Initialize labels with 'O' (outside any entity)

    # Loop through each dictionary in the 'privacy_mask' list
    for mask in example['privacy_mask']:
        if isinstance(mask, dict):  # Check if each item in the list is a dictionary
            for label_key, redacted_value in mask.items():
                label_id = label_map.get(label_key, 0)  # Get the corresponding label ID from the map, defaulting to 0

                # Find the positions of redacted_value within the unmasked_text to align labels
                start_index = example['unmasked_text'].find(redacted_value)
                if start_index != -1:
                    end_index = start_index + len(redacted_value)
                    span_tokens = tokenizer(example['unmasked_text'][start_index:end_index], add_special_tokens=False).tokens()

                    # Mark tokens within the redacted span
                    for i, token in enumerate(tokenized_inputs.tokens()):
                        if token in span_tokens:
                            labels[i] = label_id

    # Convert labels to numeric form and align with input length
    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs['input_ids']) - len(numeric_labels))

    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs

In [15]:
# Function to tokenize and align labels
def tokenize_and_align_labels_val(example):
    tokenized_inputs = tokenizer(example['source_text'], truncation=True, padding='max_length', max_length=512)
    labels = ['O'] * len(tokenized_inputs['input_ids'])

    # Adjust indexing to fit the tokenization
    example['source_text'] = example['source_text'][:512]  # Ensure source_text matches max_length

    # Create spans and assign labels
    for span in example['privacy_mask']:
        start, end, label_name = span['start'], span['end'], span['label']
        label_id = label_map.get(label_name, 0)

        span_tokens = tokenizer(example['source_text'][start:end], add_special_tokens=False).tokens()

        # Properly create and align labels (assuming BERT tokenizer)
        for i, token in enumerate(tokenized_inputs.tokens()):
            if token in span_tokens:
                labels[i] = label_id

    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs['input_ids']) - len(numeric_labels))

    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs

In [16]:
# Apply function to dataset
# encoded_dataset = train_set1_selected.map(tokenize_and_align_labels, batched=False)
encoded_dataset = train_set2.map(tokenize_and_align_labels_train, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(encoded_dataset['input_ids'])
attention_mask = torch.tensor(encoded_dataset['attention_mask'])
labels = torch.tensor(encoded_dataset['labels'])

# Load model
model = AutoModelForTokenClassification.from_pretrained('lakshyakh93/deberta_finetuned_pii')

# Define metric computation
metric = evaluate.load('accuracy')  # You may want to load relevant metrics for token classification

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)  # Note: axis=2 for token classification
    true_labels = labels != -100  # Masking out unnecessary labels

    # Flatten predictions and true_labels to compute accuracy
    flattened_predictions = predictions[true_labels]
    flattened_labels = labels[true_labels]

    results = metric.compute(references=flattened_labels, predictions=flattened_predictions)
    return results


# Tokenize validation set
small_val = val_set.shuffle(seed=123).select(range(40))
encoded_small_val = small_val.map(tokenize_and_align_labels_val, batched=False)

# Training arguments
train_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-2,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=10,
    eval_steps=10,
    gradient_accumulation_steps=2,  # Adjust if needed
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_small_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/6.31k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/555M [00:00<?, ?B/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Run training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,1.941200,0.000000,1.000000
2,0.000000,0.000000,1.000000


TrainOutput(global_step=36, training_loss=0.5392323070102267, metrics={'train_runtime': 148.0211, 'train_samples_per_second': 4.053, 'train_steps_per_second': 0.243, 'total_flos': 175710303682560.0, 'train_loss': 0.5392323070102267, 'epoch': 2.88})

In [18]:
trainer.evaluate()

{'eval_loss': 0.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 2.4517,
 'eval_samples_per_second': 16.315,
 'eval_steps_per_second': 2.039,
 'epoch': 2.88}

In [46]:
# Save model and tokenizer
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_tokenizer')

('./trained_tokenizer/tokenizer_config.json',
 './trained_tokenizer/special_tokens_map.json',
 './trained_tokenizer/vocab.json',
 './trained_tokenizer/merges.txt',
 './trained_tokenizer/added_tokens.json',
 './trained_tokenizer/tokenizer.json')

In [47]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load the trained model and tokenizer
model2 = AutoModelForTokenClassification.from_pretrained('./trained_model')
tokenizer2 = AutoTokenizer.from_pretrained('./trained_tokenizer')

In [58]:
# id2label of the model
print(model.config.id2label)

{0: 'B-PREFIX', 1: 'I-PREFIX', 2: 'B-FIRSTNAME', 3: 'I-FIRSTNAME', 4: 'B-MIDDLENAME', 5: 'B-LASTNAME', 6: 'I-LASTNAME', 7: 'O', 8: 'B-JOBDESCRIPTOR', 9: 'B-JOBTITLE', 10: 'I-JOBTITLE', 11: 'B-COMPANY_NAME', 12: 'I-COMPANY_NAME', 13: 'B-JOBAREA', 14: 'B-EMAIL', 15: 'I-EMAIL', 16: 'B-TIME', 17: 'I-TIME', 18: 'B-DATE', 19: 'I-DATE', 20: 'B-URL', 21: 'I-URL', 22: 'B-BITCOINADDRESS', 23: 'I-BITCOINADDRESS', 24: 'B-ETHEREUMADDRESS', 25: 'I-ETHEREUMADDRESS', 26: 'B-ACCOUNTNAME', 27: 'I-ACCOUNTNAME', 28: 'B-IBAN', 29: 'I-IBAN', 30: 'B-ACCOUNTNUMBER', 31: 'I-ACCOUNTNUMBER', 32: 'B-BIC', 33: 'I-BIC', 34: 'B-IPV4', 35: 'I-IPV4', 36: 'B-STREETADDRESS', 37: 'I-STREETADDRESS', 38: 'B-CITY', 39: 'I-CITY', 40: 'B-ZIPCODE', 41: 'I-ZIPCODE', 42: 'B-USERNAME', 43: 'I-USERNAME', 44: 'B-IPV6', 45: 'I-IPV6', 46: 'B-CREDITCARDNUMBER', 47: 'I-CREDITCARDNUMBER', 48: 'B-VEHICLEVIN', 49: 'I-VEHICLEVIN', 50: 'B-SUFFIX', 51: 'I-SUFFIX', 52: 'B-AMOUNT', 53: 'I-AMOUNT', 54: 'B-CURRENCY', 55: 'I-CURRENCY', 56: 'B-PAS

In [57]:
from transformers import pipeline

# Initialize a token classification pipeline
pii_pipeline = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy='max')

# Function to apply PII masking to a specific text column
def redact_column(dataset, column_name):
    redacted_texts = []

    for text in dataset[column_name]:
        # Get model predictions for the text
        predictions = pii_pipeline(text)
        print(predictions)

        # Replace PII spans with placeholders
        redacted_text = text
        for entity in sorted(predictions, key=lambda x: x['start'], reverse=True):  # Sort in reverse to avoid shifting positions
            label = entity['entity_group']
            redacted_text = redacted_text[:entity['start']] + f'[{label}]' + redacted_text[entity['end']:]

        redacted_texts.append(redacted_text)

    return redacted_texts

df = pd.DataFrame({'source_text': ['My name is Jane Doe']})

# Display the redacted text
print(redact_column(df, 'source_text'))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PREFIX', 'score': 1.0, 'word': ' My', 'start': 0, 'end': 2}, {'entity_group': 'PREFIX', 'score': 1.0, 'word': ' name', 'start': 2, 'end': 7}, {'entity_group': 'PREFIX', 'score': 1.0, 'word': ' is', 'start': 7, 'end': 10}, {'entity_group': 'PREFIX', 'score': 1.0, 'word': ' Jane', 'start': 10, 'end': 15}, {'entity_group': 'PREFIX', 'score': 1.0, 'word': ' Doe', 'start': 15, 'end': 19}]
['[PREFIX][PREFIX][PREFIX][PREFIX][PREFIX]']


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [22]:
import torch

In [48]:
# Define relevant labels for which we want to apply redaction
relevant_entity_labels = [
    'FIRSTNAME', 'LASTNAME', 'JOBTITLE', 'EMAIL', 'PHONE_NUMBER', 'ADDRESS', 'SSN', 'USERNAME', 'AGE'
]

# Updated placeholder mapping for entities (with only relevant entities)
entity_to_placeholder = {
    'FIRSTNAME': '[FIRSTNAME_1]',
    'LASTNAME': '[LASTNAME_1]',
    'JOBTITLE': '[JOBTITLE_1]',
    'EMAIL': '[EMAIL_1]',
    'PHONE_NUMBER': '[PHONE_NUMBER_1]',
    'SSN': '[SSN_1]',
    'USERNAME': '[USERNAME_1]',
    'AGE': '[AGE_1]'
}

# Ensure that the model is on the right device (CUDA if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2.to(device)

# List of labels to ignore
unwanted_labels = ['B-PREFIX', 'I-PREFIX']

# Adjust redaction process to ignore unwanted labels
def predict_entities(text):
    inputs = tokenizer2(text, return_tensors='pt', truncation=True, padding=True).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).to(device)

    tokens = tokenizer2.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_labels = [id2label[label_id.item()] for label_id in predictions[0]]

    # Debug: Print token labels to check predictions
    print("Tokens:", tokens)
    print("Predicted Labels:", predicted_labels)

    redacted_text = ''
    current_entity = ''

    for token, label in zip(tokens, predicted_labels):
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            continue  # Skip special tokens

        token_cleaned = token.lstrip('Ġ')  # Remove leading 'Ġ' from token

        if label not in unwanted_labels:  # Ignore unwanted labels like 'B-PREFIX'
            if label != 'O':  # Non-neutral labels
                entity_label = label[2:]  # Remove 'B-' or 'I-' prefix to get the entity type
                placeholder = entity_to_placeholder.get(entity_label, token_cleaned)

                if label.startswith('B-'):  # For 'B-', start a new entity
                    if current_entity:
                        redacted_text += f' {current_entity}'
                    current_entity = placeholder
                else:  # For 'I-', continue the entity
                    current_entity += f' {token_cleaned}'
            else:
                if current_entity:
                    redacted_text += f' {current_entity}'  # End the current entity
                redacted_text += f' {token_cleaned}'  # Add the token if it's not part of an entity
                current_entity = ''  # Reset current entity

    # Add any remaining entity at the end
    if current_entity:
        redacted_text += f' {current_entity}'

    return redacted_text.strip()

In [49]:
# Test with auto-generated entries
test_texts = [
    'Hello Mr. John Doe',
    'Contact Jane Doe'
]

for text in test_texts:
    print(f'\nInput text: {text}')
    redacted_text = predict_entities(text)
    print(f'Redacted output: {redacted_text}')


Input text: Hello Mr. John Doe
Tokens: ['[CLS]', 'ĠHello', 'ĠMr', '.', 'ĠJohn', 'ĠDoe', '[SEP]']
Predicted Labels: ['B-PREFIX', 'B-PREFIX', 'B-PREFIX', 'B-PREFIX', 'B-PREFIX', 'B-PREFIX', 'B-PREFIX']
Redacted output: 

Input text: Contact Jane Doe
Tokens: ['[CLS]', 'ĠContact', 'ĠJane', 'ĠDoe', '[SEP]']
Predicted Labels: ['B-PREFIX', 'B-PREFIX', 'B-PREFIX', 'B-PREFIX', 'B-PREFIX']
Redacted output: 
